In [1]:

import os
import shutil
import zipfile
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from tqdm import tqdm
import glob
import functools

# --- CONFIGURATION ---
# Kaggle Paths
MONET_PATH = '../input/gan-getting-started/monet_jpg'
PHOTO_PATH = '../input/gan-getting-started/photo_jpg'

# --- DYNAMIC WEIGHTS SEARCH ---
print("Searching for weights in ../input/ ...")
found_weights = glob.glob('../input/**/*.pth', recursive=True)

WEIGHTS_PATH = None

# Prioritize files for Experiment 3 (look for _3 suffix)
for f in found_weights:
    if 'G_Monet' in f and 'epoch' in f and '_3' in f:
        WEIGHTS_PATH = f
        break

# Fallback: take the first .pth found if no specific match
if WEIGHTS_PATH is None and len(found_weights) > 0:
    WEIGHTS_PATH = found_weights[0]

if WEIGHTS_PATH:
    print(f"✅ Found weights at: {WEIGHTS_PATH}")
else:
    print("❌ No weights found! Please check your dataset.")
    WEIGHTS_PATH = 'G_Monet_epoch_30_3.pth' # Default fallback

# Output
OUTPUT_DIR = '../tmp/images'
os.makedirs(OUTPUT_DIR, exist_ok=True)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")


Searching for weights in ../input/ ...
✅ Found weights at: ../input/g-monet-epoch-30-3-pth/G_Photo_epoch_30_3.pth
Using device: cpu


In [2]:

# --- MODELS (Inlined from src/models/generator/unet_gan.py) ---

class UNetGenerator(nn.Module):
    def __init__(self, input_nc=3, output_nc=3, num_downs=6, ngf=32):
        super(UNetGenerator, self).__init__()
        
        # Construct U-Net structure
        unet_block = UNetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=None, innermost=True)  # add the innermost layer
        
        # Reduced number of intermediate layers
        # For num_downs=6, we subtract 5 (outermost + innermost + 3 reduction layers) = 1 intermediate layer
        for i in range(num_downs - 5):
            unet_block = UNetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=unet_block)
        
        # Gradually reduce the number of filters from ngf * 8 to ngf
        unet_block = UNetSkipConnectionBlock(ngf * 4, ngf * 8, input_nc=None, submodule=unet_block)
        unet_block = UNetSkipConnectionBlock(ngf * 2, ngf * 4, input_nc=None, submodule=unet_block)
        unet_block = UNetSkipConnectionBlock(ngf, ngf * 2, input_nc=None, submodule=unet_block)
        
        self.model = UNetSkipConnectionBlock(output_nc, ngf, input_nc=input_nc, submodule=unet_block, outermost=True)  # add the outermost layer

    def forward(self, x):
        return self.model(x)

class UNetSkipConnectionBlock(nn.Module):
    def __init__(self, outer_nc, inner_nc, input_nc=None, submodule=None, outermost=False, innermost=False, norm_layer=nn.InstanceNorm2d, use_dropout=False):
        super(UNetSkipConnectionBlock, self).__init__()
        self.outermost = outermost
        if type(norm_layer) == functools.partial:
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d

        if input_nc is None:
            input_nc = outer_nc
        
        downconv = nn.Conv2d(input_nc, inner_nc, kernel_size=4, stride=2, padding=1, bias=use_bias)
        downrelu = nn.LeakyReLU(0.2, True)
        downnorm = norm_layer(inner_nc)
        uprelu = nn.ReLU(True)
        upnorm = norm_layer(outer_nc)

        if outermost:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc, kernel_size=4, stride=2, padding=1)
            down = [downconv]
            up = [uprelu, upconv, nn.Tanh()]
            model = down + [submodule] + up
        elif innermost:
            upconv = nn.ConvTranspose2d(inner_nc, outer_nc, kernel_size=4, stride=2, padding=1, bias=use_bias)
            down = [downrelu, downconv]
            up = [uprelu, upconv, upnorm]
            model = down + up
        else:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc, kernel_size=4, stride=2, padding=1, bias=use_bias)
            down = [downrelu, downconv, downnorm]
            up = [uprelu, upconv, upnorm]

            if use_dropout:
                model = down + [submodule] + up + [nn.Dropout(0.5)]
            else:
                model = down + [submodule] + up

        self.model = nn.Sequential(*model)

    def forward(self, x):
        if self.outermost:
            return self.model(x)
        else:   # add skip connections
            return torch.cat([x, self.model(x)], 1)


In [3]:

# --- DATASET & TRANSFORMS ---

class ImageDataset(Dataset):
    def __init__(self, files, transform=None):
        self.files = files
        self.transform = transform

    def __getitem__(self, index):
        img_path = self.files[index]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, os.path.basename(img_path)

    def __len__(self):
        return len(self.files)

def get_transforms():
    # Only resize and normalize for inference
    return transforms.Compose([
        transforms.Resize((256, 256), Image.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

def denormalize(tensor):
    return tensor * 0.5 + 0.5


In [4]:

# --- MAIN EXECUTION ---

# 1. Initialize Model
# Experiment 3 used U-Net with ngf=32, num_downs=6
model = UNetGenerator(num_downs=6, ngf=32).to(DEVICE)

# 2. Load Weights
if os.path.exists(WEIGHTS_PATH):
    print(f"Loading weights from {WEIGHTS_PATH}...")
    state_dict = torch.load(WEIGHTS_PATH, map_location=DEVICE)
    model.load_state_dict(state_dict)
else:
    raise FileNotFoundError(f"Could not find weight file at {WEIGHTS_PATH}")

model.eval()

# 3. Prepare Data
import glob
photo_files = sorted(glob.glob(os.path.join(PHOTO_PATH, "*.jpg")))
print(f"Found {len(photo_files)} photo images.")

# 4. Generate Images
dataset = ImageDataset(photo_files, transform=get_transforms())
loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

print("Starting generation...")
cnt = 0

with torch.no_grad():
    for img_tensor, filename in tqdm(loader):
        img_tensor = img_tensor.to(DEVICE)
        
        # Generate
        generated = model(img_tensor)
        
        # Post-process
        generated = denormalize(generated).cpu()
        
        # Save images
        # We need to save as jpg
        for i in range(generated.size(0)):
            img_np = generated[i].permute(1, 2, 0).numpy()
            img_np = (img_np * 255).astype(np.uint8)
            img_pil = Image.fromarray(img_np)
            
            # Save to temporary directory
            save_path = os.path.join(OUTPUT_DIR, filename[i])
            img_pil.save(save_path)
            cnt += 1

print(f"Generated {cnt} images at {OUTPUT_DIR}")


Loading weights from ../input/g-monet-epoch-30-3-pth/G_Photo_epoch_30_3.pth...
Found 7038 photo images.
Starting generation...


100%|██████████| 7038/7038 [07:25<00:00, 15.80it/s]

Generated 7038 images at ../tmp/images


In [6]:

# --- ZIP RESULTS ---
import shutil

# The competition requires a file named 'images.zip' in the working directory
print("Zipping images...")
shutil.make_archive('images', 'zip', OUTPUT_DIR)
print("✅ Created images.zip")

# Verify
if os.path.exists("images.zip"):
    print(f"images.zip created successfully. Size: {os.path.getsize('images.zip') / 1024 / 1024:.2f} MB")
else:
    print("❌ Failed to create images.zip")

# Optional: Clean up temp folder to save space/inodes if needed
# shutil.rmtree(OUTPUT_DIR)


Zipping images...
✅ Created images.zip
images.zip created successfully. Size: 90.35 MB
